<a href="https://colab.research.google.com/github/pristineVedansh/q-DPSGD/blob/main/qDPSGD_EMNIST_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Images

In [ ]:
import numpy as np
from keras.datasets import mnist, cifar10, cifar100 
from keras.utils import to_categorical
import keras
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

!pip install tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

     |████████████████████████████████| 194kB 14.1MB/s 


In [ ]:
#@title Default title text
learning_rate = 0.15
noise_multiplier = 0.6
l2_norm_clip = 1.0
batch_size = 128
epochs = 100
microbatches = 128

Can switch among cifar10 | cifar100 | mnist

In [ ]:
#(x_train, y_train_), (x_test, y_test_) = cifar10.load_data()
#(x_train, y_train_), (x_test, y_test_) = cifar100.load_data()
(x_train, y_train_), (x_test, y_test_) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
if( x_train.ndim < 4 ):
    x_train = np.expand_dims( x_train, -1 )
    x_test  = np.expand_dims( x_test,  -1 )

In [ ]:
y_train = to_categorical(y_train_)
y_test = to_categorical(y_test_)

## q-Neuron

In [ ]:
import keras
import keras.backend as K                                                                                                                                                             

class Qact( keras.engine.Layer ):

    def __init__( self, act, init_stddev=1., **kwargs ):
        # init_stddev is the scale of randomness
        
        super( Qact, self ).__init__( **kwargs )

        self.supports_masking = True
        self.eps              = 1e-3

        self.act              = act
        self.init_stddev      = init_stddev

    def build( self, input_shape ):
        super( Qact, self ).build( input_shape )

    def call( self, inputs ):
        _stddev = self.init_stddev
        Q = K.random_normal( shape=K.shape(inputs) )
        Q = ( 2*K.cast( Q>=0, 'float32' )-1 ) * ( _stddev * K.abs(Q) + self.eps )

        return ( self.act( inputs * (1+Q) ) - self.act( inputs ) ) / Q

    def compute_output_shape( self, input_shape ):
        return input_shape

## Compute Epsilon

In [ ]:
"""
def compute_epsilon(steps):
    Computes epsilon value for given hyperparameters."
    if noise_multiplier == 0.0:
        return float('inf')
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
    sampling_probability = batch_size / 60000
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)
    # Delta is set to 1e-5 because MNIST has 60000 training points.
    return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
"""

'\ndef compute_epsilon(steps):\n    Computes epsilon value for given hyperparameters."\n    if noise_multiplier == 0.0:\n        return float(\'inf\')\n    orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))\n    sampling_probability = batch_size / 60000\n    rdp = compute_rdp(q=sampling_probability,\n                    noise_multiplier=noise_multiplier,\n                    steps=steps,\n                    orders=orders)\n    # Delta is set to 1e-5 because MNIST has 60000 training points.\n    return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]\n'

## CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [ ]:
optimizer = DPKerasSGDOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=microbatches,
        learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, 
                kernel_size=(3, 3),
                activation='sigmoid',
                input_shape=x_train.shape[1:]))

model.add(MaxPool2D())
model.add(Conv2D(filters=64,
                kernel_size=(3, 3),
                activation='sigmoid'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:

qmodel = Sequential()
qmodel.add(Conv2D(filters=32, 
                kernel_size=(3, 3),
                input_shape=x_train.shape[1:]))
qmodel.add(Qact(keras.activations.sigmoid))

qmodel.add(MaxPool2D())
qmodel.add(Conv2D(filters=64,
                kernel_size=(3, 3)))
qmodel.add(Qact(keras.activations.sigmoid))

qmodel.add(MaxPool2D())
qmodel.add(Flatten())
qmodel.add(Dense(y_train.shape[1], activation='softmax'))
qmodel.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Training

## Plot the training/validation curves

In [ ]:
history  =  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.20)

Epoch 1/100
375/375 [==============================] - 66s 127ms/step - loss: 2.3143 - accuracy: 0.1059 - val_loss: 2.3088 - val_accuracy: 0.0769
Epoch 2/100
375/375 [==============================] - 47s 126ms/step - loss: 2.3068 - accuracy: 0.1059 - val_loss: 2.3021 - val_accuracy: 0.0995
Epoch 3/100
375/375 [==============================] - 48s 127ms/step - loss: 2.3027 - accuracy: 0.1114 - val_loss: 2.2972 - val_accuracy: 0.1060
Epoch 4/100
375/375 [==============================] - 48s 127ms/step - loss: 2.2984 - accuracy: 0.1257 - val_loss: 2.2922 - val_accuracy: 0.2685
Epoch 5/100
375/375 [==============================] - 48s 127ms/step - loss: 2.2931 - accuracy: 0.1311 - val_loss: 2.2887 - val_accuracy: 0.1067
Epoch 6/100
375/375 [==============================] - 47s 127ms/step - loss: 2.2840 - accuracy: 0.1414 - val_loss: 2.2781 - val_accuracy: 0.0998
Epoch 7/100
375/375 [==============================] - 47s 125ms/step - loss: 2.2675 - accuracy: 0.1753 - val_loss: 2.2447 -

In [ ]:
qhistory = qmodel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.20)

In [ ]:
import matplotlib.pyplot as plt

loss  = history.history['loss']
val   = history.history['val_accuracy']
qloss = qhistory.history['loss']
qval  = qhistory.history['val_accuracy']


epochs = range(1, len(history.history['accuracy']) + 1)
qepochs = range(1, len(qhistory.history['accuracy']) + 1)

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 10), dpi=100)
ax1.plot(epochs, loss,  'bo', ls="--", label='Training loss')
ax1.plot(qepochs, qloss, 'ro', ls="--", label="Training loss (q-DPSGD)")

ax2 = ax1.twinx()
ax2.plot(epochs, val,   'bd', ls="-.", label='Validation acc')
ax2.plot(qepochs, qval,  'rd', ls="-.", label='Validation acc (q-DPSGD)')

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax2.set_ylabel('Accuracy')
ax1.legend(loc=1)
ax2.legend(loc=5)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 10), dpi=100)
#fig.suptitle('Training on MNIST', fontsize=20)
ax1.plot(epochs, loss,  'bo', ls="--", label='Training loss (q-DPSGD)')
ax1.plot(qepochs, qloss, 'ro', ls="--", label="Training loss")

ax2 = ax1.twinx()
ax2.plot(epochs, val,   'bd', ls="-.", label='Validation acc (q-DPSGD)')
ax2.plot(qepochs, qval,  'rd', ls="-.", label='Validation acc')

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax2.set_ylabel('Accuracy')
ax1.legend(loc=10)
ax2.legend(loc=5)
plt.show()

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
qmodel.evaluate(x=x_test, y=y_test)